In [ ]:
import sys
import os
from pathlib import Path

# For Jupyter or interactive use — use current working directory as script base
notebook_path = Path().resolve()

# Assume notebook is in a subfolder of the repo — go up one level
file_dir = notebook_path.parent.parent

# Set working directory to the repo root
os.chdir(file_dir)
print("Working directory set to:", Path.cwd())

sys.path.append(str(file_dir))

import pandas as pd
import numpy as np
import nibabel as nib
from statsmodels.stats.multitest import multipletests
from netneurotools import stats

data_dir = f'{file_dir}/data'


Working directory set to: /Users/melinatsotras/Desktop/submission


In [ ]:
# Load cell abundance data and set 'D99' as the index
cells = pd.read_csv(f'{data_dir}/level_2_cell_abundance.csv')
cells = cells.set_index('D99')

# Extract names of cells
cell_names = cells.columns.tolist()

# Load and add  total similarity strength metric to cells dataframe
cells['total_similarity_strength'] = pd.read_csv(f'{file_dir}/MIND_Network/total_similarity_strength.csv')\
    .set_index('region')['total_similarity_strength']

# Load and add age effects metric to cells dataframe
cells['age_effects']  = pd.read_csv(f'{file_dir}/Mixed_Effects_Models/regionwise_age_effects_MixedLM.csv')\
    .set_index('region')['t_value']


# Load D99 to Yeo network labels (skip first row if header is repeated or unwanted)
d99_labels = pd.read_csv(f'{data_dir}/d99_to_yeo_network_labels.csv').iloc[1:]

# Merge cell data with network labels on 'D99' to match regions to yeo networks
cells = cells.merge(d99_labels[['D99', 'yeo_label']], on='D99', how='inner')

# Fetch D99 parcellation labels for surface regions
cells_on_surf = pd.DataFrame({'D99': nib.load(f'{data_dir}/gifti/CIVET_macaque-alpha-0.2/D99_atlas_rsl_sym_left.label.gii').agg_data()})

# Merge surface labels with the cells dataframe (left join to keep all surface regions)
cells_on_surf = cells_on_surf.merge(cells, how='left', on='D99')


Cell names: ['ASC', 'EC', 'MG', 'OLG', 'OPC', 'VLMC', 'L2', 'L2_3', 'L2_3_4', 'L3_4', 'L3_4_5', 'L4', 'L4_5', 'L4_5_6', 'L5_6', 'L6', 'LAMP5', 'PVALB', 'PV_CHC', 'RELN', 'SST', 'VIP', 'VIP_RELN']


In [ ]:
## Get spherical coordinates & spin them to get new rotations


lhsphere = f'{data_dir}/gifti/CIVET_macaque-alpha-0.2/sphere_left_iso.surf.gii'
rhsphere = f'{data_dir}/gifti/CIVET_macaque-alpha-0.2/sphere_right_iso.surf.gii'

spherical_vertices_lh = nib.load(lhsphere).agg_data('pointset')
spherical_vertices_rh = nib.load(rhsphere).agg_data('pointset')
vertices = np.concatenate([spherical_vertices_lh, spherical_vertices_rh])

hemiid = [0]*40962 +[1]*40962



In [ ]:
for i,cell in enumerate(cell_names):
        print(cell)
        # spin vertices
        spins_vertices = stats.gen_spinsamples(vertices, hemiid, n_rotate = 5000, seed =i)
        spins_vertices = spins_vertices[:40962] # LH only

        cell_array = cells_on_surf[cell].to_numpy()
        null_map = pd.DataFrame(cell_array[spins_vertices])
        null_map['yeo_label'] = cells_on_surf.yeo_label
        
        pd.DataFrame(null_map).to_csv(f'{file_dir}/Enrichment_Analysis/macaque/output/NULL_vertexwise_{cell}_5000.csv')

ASC
EC
MG
OLG
OPC
VLMC
L2
L2_3
L2_3_4
L3_4
L3_4_5
L4
L4_5
L4_5_6
L5_6
L6
LAMP5
PVALB
PV_CHC
RELN
SST
VIP
VIP_RELN


In [10]:
# Create a lookup table of D99 regions and their associated Yeo network labels
d99_yeo_lookup = cells[['D99', 'yeo_label']].copy()

# Create a mapping from each unique Yeo label to a unique integer (starting from 1)
# Sorting ensures consistent and ordered label-to-number assignment
unique_yeo_labels = d99_yeo_lookup['yeo_label'].sort_values().unique()
yeo_dict = {label: idx + 1 for idx, label in enumerate(unique_yeo_labels)}

# Map each Yeo label to its corresponding number using the dictionary
d99_yeo_lookup['yeo_num'] = d99_yeo_lookup['yeo_label'].map(yeo_dict)

# Display the resulting lookup table with numeric labels
d99_yeo_lookup


,D99,yeo_label,yeo_num
0,2,Default Mode,1
1,3,Limbic,4
2,5,Limbic,4
3,7,Ventral Attention,6
4,8,Limbic,4
...,...,...,...
136,188,Limbic,4
137,194,Limbic,4
138,195,Limbic,4
139,218,Frontoparietal,3


In [ ]:
means = pd.DataFrame(0, index = yeo_dict.keys(), columns = cell_names)
sds = pd.DataFrame(0, index = yeo_dict.keys(), columns = cell_names)
pvals = pd.DataFrame(0, index = yeo_dict.keys(), columns = cell_names)
zscores = pd.DataFrame(0, index = yeo_dict.keys(), columns = cell_names)
true_value = cells[cell_names+['yeo_label']].groupby('yeo_label').mean()

for cell in cell_names:
    null = pd.read_csv(f"{file_dir}/Enrichment_Analysis/macaque/output/NULL_vertexwise_{cell}.csv").groupby('yeo_label').mean()

    means[cell] = null.mean(axis = 1, skipna = True)
    sds[cell] = null.std(axis = 1, skipna = True)

    zscored_null = null.apply(lambda row: (row-means[cell][row.name])/sds[cell][row.name], axis=1)
    zscores[cell] = (true_value[cell]-means[cell])/sds[cell]
    pvals[cell] = zscored_null.abs().apply(lambda row: (row > zscores.abs()[cell][row.name]).sum(), axis=1)/null.shape[1]



In [21]:
pvalues = pvals.copy()
zscores = zscores.T
pvalues = pvalues.T

In [ ]:
fdr_p_value = pvalues.copy()

for col in pvalues.columns:
    _, fdr_corrected_p, _, _ = multipletests(pvalues[col], method='fdr_bh')
    fdr_p_value[col] = fdr_corrected_p
display(fdr_p_value)

fdr_p_value.to_csv(f'{file_dir}/Enrichment_Analysis/macaque/enrichment_analysis_fdr_pvalues_macaque_5000maps.csv')

,Default Mode,Dorsal Attention,Frontoparietal,Limbic,SomatoMotor,Ventral Attention,Visual
ASC,0.125638,0.8786,0.834133,0.319043,0.409400,0.060457,0.473800
EC,0.071300,0.8786,0.834133,0.319043,0.343850,0.858109,0.473800
MG,0.000000,0.8786,0.834133,0.000000,0.718855,0.041400,0.473800
OLG,0.125638,0.8786,0.834133,0.363707,0.718855,0.504376,0.951000
OPC,0.018400,0.8786,0.834133,0.000000,0.718855,0.163760,0.246100
VLMC,0.003680,0.8786,0.834133,0.262200,0.718855,0.865800,0.563677
L2,0.776524,0.8786,0.834133,0.266340,0.852600,0.104650,0.000000
L2_3,0.125638,0.8786,0.538200,0.004600,0.409400,0.118067,0.000000
L2_3_4,0.250095,0.8786,0.538200,0.046920,0.718855,0.604053,0.942791
L3_4,0.431480,0.8786,0.759000,0.116314,0.343850,0.004600,0.563677


In [ ]:
display(zscores)
zscores.to_csv(f'{file_dir}/Enrichment_Analysis/macaque/enrichment_analysis_zscores_macaque_5000maps.csv')

,Default Mode,Dorsal Attention,Frontoparietal,Limbic,SomatoMotor,Ventral Attention,Visual
ASC,1.658145,0.263114,0.768750,1.346785,1.558740,2.013131,-1.299070
EC,2.193893,-1.189224,-0.418461,1.374291,1.735689,-0.272734,-1.594866
MG,2.779995,-0.671872,0.440301,3.179816,1.147263,2.156988,-1.413760
OLG,-1.724670,0.770119,0.294050,-1.138006,0.831530,-0.886900,-0.094142
OPC,2.193631,-0.500877,0.735856,2.297959,0.916071,1.712318,-1.697831
VLMC,4.068769,-0.502690,-0.752696,1.316956,0.553193,-0.112967,-1.274484
L2,0.414911,0.517022,-0.877016,1.602467,-0.226608,2.018826,2.877603
L2_3,-1.797949,-0.300302,2.089183,-4.323889,-1.595742,-2.019686,2.957493
L2_3_4,1.195320,-0.559348,-2.259902,2.983315,-0.479724,0.661648,-0.182188
L3_4,-0.908495,-0.563599,1.707401,-2.111477,1.890190,-3.513859,-1.074994
